# Filter Application

> The raw files were all gathered using [these queries](./2024-06-18T20-19-02_queries.md)
> on June 18th, 2024 at roughly 8pm using access via Ulm University's VPN.

This file illustrates the filters used before abstract screening. The applied
filters are only able to filter out matches that are marked correctly and
provide the accurate and sufficient metadata. As this is not the case,
additional manual filtering is required after running this script.

In [1]:
import numpy as np
import pandas as pd
import rispy
import bibtexparser
from collections import defaultdict
import re

## Loading Files

In [2]:
def load_bib(path: str) -> pd.DataFrame:
  '''Loads search results from a .bib file into a pandas DataFrame'''
  with open(path, 'r', encoding='utf-8') as file:
    bib_data = bibtexparser.load(file)
  return pd.DataFrame(bib_data.entries)


def load_ris(path: str) -> pd.DataFrame:
  '''Loads search results from a .ris file into a pandas DataFrame'''
  with open(path, 'r', encoding='utf-8') as file:
    ris_data = rispy.load(file)
  return pd.DataFrame(ris_data)

In [3]:
# load ACM Digital Library results
acm_df = load_bib('./data/raw/acm_digitallibrary.bib')
print(f'Loaded {acm_df.shape[0]} entries from ACM Digital Library')

# load APA PsycInfo results
apa_df = load_ris('./data/raw/apa_psycinfo.ris')
print(f'Loaded {apa_df.shape[0]} entries from APA PsycInfo')

# load IEEE Xplore results
ieee_df = load_ris('./data/raw/ieee_xplore_fromDOIs.ris')
print(f'Loaded {ieee_df.shape[0]} entries from IEEE Xplore')

# load PubMed results (with extra abstract data from PubMed2XLSX)
pubmed_df = pd.read_csv('./data/raw/pubmed_base.csv')
pubmed2xl_df = pd.read_csv('./data/raw/pd2xl.csv')
pubmed_df.set_index('PMID', inplace=True)
pubmed2xl_df.set_index('PMID', inplace=True)
pubmed_df['Abstract'] = pubmed2xl_df['Abstract']
print(f'Loaded {pubmed_df.shape[0]} entries from PubMed & MEDLINE')

# load Web of Science results
wos_df = load_bib('./data/raw/webofscience_0001-1000.bib')
wos_df = pd.concat([wos_df, load_bib('./data/raw/webofscience_1001-1807.bib')])
print(f'Loaded {wos_df.shape[0]} entries from Web of Science CORE')

# load Google Scholar results
gs_df = pd.read_csv('./data/raw/googlescholar_appended.csv')
print(f'Loaded {gs_df.shape[0]} entries from Google Scholar')

Loaded 943 entries from ACM Digital Library
Loaded 1117 entries from APA PsycInfo
Loaded 204 entries from IEEE Xplore
Loaded 2911 entries from PubMed & MEDLINE
Loaded 1807 entries from Web of Science CORE
Loaded 300 entries from Google Scholar


In [4]:
def get_num_entries() -> int:
  '''Returns the number of elements across all loaded dataframes'''
  dfs: list[pd.DataFrame] = [acm_df, apa_df, ieee_df, pubmed_df, wos_df, gs_df]
  return sum(map(lambda x: x.shape[0], dfs))

In [5]:
print('-----------------------------------------------------------------------')
print(f'Loaded {get_num_entries()} across all databases using this search strategy')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Loaded 7282 across all databases using this search strategy
-----------------------------------------------------------------------


## Export Functions

In [6]:
def export_to_bib(df: pd.DataFrame, fname: str) -> None:
  '''Exports the contents of a dataframe to a bib file'''
  df = df.map(str)  # convert all values to str, as bib files need them
  bib_db = bibtexparser.bibdatabase.BibDatabase()  # BibTex file structure
  bib_db.entries = df.to_dict(orient='records')  # set the records as contents
  writer = bibtexparser.bwriter.BibTexWriter()
  with open(fname, 'w', encoding='utf-8') as file:
    file.write(writer.write(bib_db))

def export_to_ris(df: pd.DataFrame, fname: str) -> None:
  '''Exports the contents of a DataFrame to an RIS file'''
  # convert all values to strings, as RIS files expect flattened strings
  df = df.map(lambda x: x if isinstance(x, dict) or isinstance(x, list) else '' if pd.isna(x) else str(x))
  with open(fname, 'w', encoding='utf-8') as file:
    rispy.dump(df.to_dict(orient='records'), file)


def export_to_csv(df: pd.DataFrame, fname: str) -> None:
  '''Exports the contents of a DataFrame to a CSV file'''
  df.to_csv(fname, index=False)


def get_filtered_out_entries(df: pd.DataFrame,
                             df_filtered: pd.DataFrame) -> pd.DataFrame:
  '''Given the original dataframe and the filtered dataframe this returns a
     dataframe containing only the filtered out entries'''
  return df[~df.index.isin(df_filtered.index)]

## Filter 1: Date

The first filter removes all publications that resulted in a match for the query
but cannot realistically include mobile application systems. The reason for that
assumption is that neither the original iPhone nor the first Android-based
smartphone were released before autumn 2008 and were realistically not available
before the year 2009.

It is assumed that no research groups got early access to the iPhone for their
research as researchers were not the target audience for the first smartphones
but rather the general public.

In the case of Google Scholar, five matches have no year information attributed
to them. Manual inspection reveals, that there are versions of those matches,
that were released after 2009. Therefore, none of those entries are removed.

In [7]:
num_before = get_num_entries()  # number of entries before applying filter
cutoff_year = 2009

In [8]:
# apply year filter on ACM Digital Library
acm_df_filtered = acm_df[acm_df['year'].astype(int) >= cutoff_year]
acm_df_excluded = get_filtered_out_entries(acm_df, acm_df_filtered)
print(f'Removed {(acm_df_excluded.shape[0])} publications from ACM Digital Library')
export_to_bib(acm_df_filtered, './data/filter1_Date/acm_filter_date_in.bib')
export_to_bib(acm_df_excluded, './data/filter1_Date/acm_filter_date_out.bib')
acm_df = acm_df_filtered

Removed 23 publications from ACM Digital Library


In [9]:
# apply year filter on APA PsycInfo
apa_df_filtered = apa_df[apa_df['publication_year'].str.rstrip('//').astype(int) >= cutoff_year]
apa_df_excluded = get_filtered_out_entries(apa_df, apa_df_filtered)
print(f'Removed {(apa_df_excluded.shape[0])} publications from APA PsycInfo')
export_to_ris(apa_df_filtered, './data/filter1_Date/apa_filter_date_in.ris')
export_to_ris(apa_df_excluded, './data/filter1_Date/apa_filter_date_out.ris')
apa_df = apa_df_filtered

Removed 4 publications from APA PsycInfo


In [10]:
# apply year filter on IEEE Xplore
ieee_df_filtered = ieee_df[ieee_df['year'].astype(int) >= cutoff_year]
ieee_df_excluded = get_filtered_out_entries(ieee_df, ieee_df_filtered)
print(f'Removed {(ieee_df_excluded.shape[0])} publications from IEEE Xplore')
export_to_ris(ieee_df_filtered, './data/filter1_Date/ieee_filter_date_in.ris')
export_to_ris(ieee_df_excluded, './data/filter1_Date/ieee_filter_date_out.ris')
ieee_df = ieee_df_filtered

Removed 7 publications from IEEE Xplore


In [11]:
# apply year filter on PubMed & MEDLINE
pubmed_df_filtered = pubmed_df[pubmed_df['Publication Year'].astype(int) >= cutoff_year]
pubmed_df_excluded = get_filtered_out_entries(pubmed_df, pubmed_df_filtered)
print(f'Removed {(pubmed_df_excluded.shape[0])} publications from PubMed & MEDLINE')
export_to_csv(pubmed_df_filtered, './data/filter1_Date/pubmed_filter_date_in.csv')
export_to_csv(pubmed_df_excluded, './data/filter1_Date/pubmed_filter_date_out.csv')
pubmed_df = pubmed_df_filtered

Removed 5 publications from PubMed & MEDLINE


In [12]:
# apply year filter on Web of Science
wos_df_filtered = wos_df[wos_df['year'].astype(int) >= cutoff_year]
wos_df_excluded = get_filtered_out_entries(wos_df, wos_df_filtered)
print(f'Removed {(wos_df_excluded.shape[0])} publications from Web of Science CORE')
export_to_bib(wos_df_filtered, './data/filter1_Date/wos_filter_date_in.bib')
export_to_bib(wos_df_excluded, './data/filter1_Date/wos_filter_date_out.bib')
wos_df = wos_df_filtered

Removed 1 publications from Web of Science CORE


In [13]:
# apply year filter on Google Scholar
gs_df['Year'] = gs_df['Year'].fillna(5000)  # to keep nan values
gs_df_filtered = gs_df[gs_df['Year'] >= cutoff_year]
gs_df_filtered['Year'] = gs_df_filtered['Year'].replace(5000, np.nan)
gs_df_excluded = get_filtered_out_entries(gs_df, gs_df_filtered)
print(f'Removed {(gs_df_excluded.shape[0])} publications from Google Scholar')
export_to_csv(gs_df_filtered, './data/filter1_Date/gs_filter_date_in.csv')
export_to_csv(gs_df_excluded, './data/filter1_Date/gs_filter_date_out.csv')
gs_df = gs_df_filtered

Removed 0 publications from Google Scholar


In [14]:
print('-----------------------------------------------------------------------')
print(f'Removed a total of {num_before - get_num_entries()} publications that were released before {cutoff_year}')
print(f'  => {get_num_entries()} entries left in total')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Removed a total of 44 publications that were released before 2009
  => 7238 entries left in total
-----------------------------------------------------------------------


## Filter 2: Language

This filter makes sure to only include publications that are written in English.
Being written in English is the only metric we have to affirm the publication
was meant for a broader and international audience.

Filtering titles can be done programatically. Filtering abstracts has to be done
manually, as not all matches have abstracts in the result set due to some
specifics in the databases such as also matching proceedings i.e. collections
of papers for search queries (if a match contains multiple papers, they usually
do not provide an abstract).

Using language detection tools such as langdetect results in a couple of false
positives, rendering them unusable. Therefore, this filter only goes over the
provided metadata and removes all entries that are marked as written in a
language other then English.

In the case of APA PsycInfo, many matches are not labeled correctly. To not
falsely remove any English matches, all matches with no language label are
assumed to be written in English, even if that is not necessarily true.

In [15]:
num_before = get_num_entries()  # number of entries before applying filter

In [16]:
print(acm_df.shape[0],
      apa_df.shape[0],
      ieee_df.shape[0],
      pubmed_df.shape[0],
      wos_df.shape[0],
      gs_df.shape[0])

920 1113 197 2906 1802 300


In [17]:
# ACM Digital Library does not provide metadata for this
acm_df_filtered = acm_df
acm_df_excluded = get_filtered_out_entries(acm_df, acm_df_filtered)
print(f'Removed {(acm_df_excluded.shape[0])} publications from ACM Digital Library')
export_to_bib(acm_df_filtered, './data/filter2_Language/acm_filter_lang_in.bib')
export_to_bib(acm_df_excluded, './data/filter2_Language/acm_filter_lang_out.bib')

Removed 0 publications from ACM Digital Library


In [18]:
# read internal lang classification on APA PsycInfo
apa_df['language'] = apa_df['language'].fillna('English')  # to keep nan values
apa_df_filtered = apa_df[apa_df['language'] == 'English']
apa_df_excluded = get_filtered_out_entries(apa_df, apa_df_filtered)
print(f'Removed {(apa_df_excluded.shape[0])} publications from APA PsycInfo')
export_to_ris(apa_df_filtered, './data/filter2_Language/apa_filter_lang_in.ris')
export_to_ris(apa_df_excluded, './data/filter2_Language/apa_filter_lang_out.ris')
apa_df = apa_df_filtered

Removed 12 publications from APA PsycInfo


In [19]:
# IEEE Xplore only publishes English articles
ieee_df_filtered = ieee_df
ieee_df_excluded = get_filtered_out_entries(ieee_df, ieee_df_filtered)
print(f'Removed {ieee_df_excluded.shape[0]} publications from IEEE Xplore')
export_to_ris(ieee_df_filtered, './data/filter2_Language/ieee_filter_lang_in.ris')
export_to_ris(ieee_df_excluded, './data/filter2_Language/ieee_filter_lang_out.ris')

Removed 0 publications from IEEE Xplore


In [20]:
# PubMed & MEDLINE do not provide metadata for this
pubmed_df_filtered = pubmed_df
pubmed_df_excluded = get_filtered_out_entries(pubmed_df, pubmed_df_filtered)
print(f'Removed {(pubmed_df_excluded.shape[0])} publications from PubMed & MEDLINE')
export_to_csv(pubmed_df_filtered, './data/filter2_Language/pubmed_filter_lang_in.csv')
export_to_csv(pubmed_df_excluded, './data/filter2_Language/pubmed_filter_lang_out.csv')

Removed 0 publications from PubMed & MEDLINE


In [21]:
# read internal lang classification on Web of Science
wos_df_filtered = wos_df[wos_df['language'] == 'English']
wos_df_excluded = get_filtered_out_entries(wos_df, wos_df_filtered)
print(f'Removed {(wos_df_excluded.shape[0])} publications from Web of Science CORE')
export_to_bib(wos_df_filtered, './data/filter2_Language/wos_filter_lang_in.bib')
export_to_bib(wos_df_excluded, './data/filter2_Language/wos_filter_lang_out.bib')
wos_df = wos_df_filtered

Removed 0 publications from Web of Science CORE


In [22]:
# Google Scholar does not provide metadata for this
gs_df_filtered = gs_df
gs_df_excluded = get_filtered_out_entries(gs_df, gs_df_filtered)
print(f'Removed {(gs_df_excluded.shape[0])} publications from Google Scholar')
export_to_csv(gs_df_filtered, './data/filter2_Language/gs_filter_lang_in.csv')
export_to_csv(gs_df_excluded, './data/filter2_Language/gs_filter_lang_out.csv')

Removed 0 publications from Google Scholar


In [23]:
print('-----------------------------------------------------------------------')
print(f'Removed a total of {num_before - get_num_entries()} publications that were not written fully in English')
print(f'  => {get_num_entries()} entries left in total')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Removed a total of 25 publications that were not written fully in English
  => 7213 entries left in total
-----------------------------------------------------------------------


In [26]:
print(acm_df.shape[0],
      apa_df.shape[0],
      ieee_df.shape[0],
      pubmed_df.shape[0],
      wos_df.shape[0],
      gs_df.shape[0])
# 920 1113 197 2906 1802 300
#     12            13

920 1101 197 2906 1789 300


## Filter 3: Publication Type

A lot of matches contain proceedings, teaching books or other collections of
papers. These are not standalone publications and therefore should not be
relevant for a scoping review.

Some entries are marked as `@book` inside bib files. These however, are not
necessarily books, as many of them look like chapters or standalone articles
instead. Therefore, this filter goes over all databases that have some sort of
entry type in their metadata and removes all of them that are defined as being
some sort of collection of papers.

In [22]:
num_before = get_num_entries()

In [23]:
# print(acm_df['ENTRYTYPE'].value_counts())
acm_df_filtered = acm_df[acm_df['ENTRYTYPE'].isin(['inproceedings', 'article', 'inbook', 'incollection'])]
acm_df_excluded = get_filtered_out_entries(acm_df, acm_df_filtered)
print(f'Removed {(acm_df_excluded.shape[0])} publications from ACM Digital Library')
export_to_bib(acm_df_filtered, './data/filter3_PublicationType/acm_filter_pubtype_in.bib')
export_to_bib(acm_df_excluded, './data/filter3_PublicationType/acm_filter_pubtype_out.bib')
acm_df = acm_df_filtered

Removed 175 publications from ACM Digital Library


In [24]:
# all matches in APA PsycInfo are marked as JOUR i.e. journal entry
print('Removed 0 publications from APA PsycInfo')
export_to_ris(apa_df, './data/filter3_PublicationType/apa_filter_pubtype_in.ris')
export_to_ris(get_filtered_out_entries(apa_df, apa_df), './data/filter3_PublicationType/apa_filter_pubtype_out.ris')

Removed 0 publications from APA PsycInfo


In [25]:
# IEEE Xplore does not provide metadata for this, but also only publishes 
# standalone articles in their IEEE Proceedings
print('Removed 0 publications from IEEE Xplore')
export_to_ris(ieee_df, './data/filter3_PublicationType/ieee_filter_pubtype_in.ris')
export_to_ris(get_filtered_out_entries(ieee_df, ieee_df), './data/filter3_PublicationType/ieee_filter_pubtype_out.ris')

Removed 0 publications from IEEE Xplore


In [26]:
# PubMed & MEDLINE do not provide metadata for this, but all publications are
# available as single paper PDF so they must be standalone
print('Removed 0 publications from PubMed & MEDLINE')
export_to_csv(pubmed_df, './data/filter3_PublicationType/pubmed_filter_pubtype_in.csv')
export_to_csv(get_filtered_out_entries(pubmed_df, pubmed_df), './data/filter3_PublicationType/pubmed_filter_pubtype_out.csv')

Removed 0 publications from PubMed & MEDLINE


In [27]:
# print(wos_df['ENTRYTYPE'].value_counts())
wos_df_filtered = wos_df[wos_df['ENTRYTYPE'].isin(['inproceedings', 'article', 'inbook', 'incollection'])]
wos_df_excluded = get_filtered_out_entries(wos_df, wos_df_filtered)
print(f'Removed {(wos_df_excluded.shape[0])} publications from Web of Science CORE')
export_to_bib(wos_df_filtered, './data/filter3_PublicationType/wos_filter_pubtype_in.bib')
export_to_bib(wos_df_excluded, './data/filter3_PublicationType/wos_filter_pubtype_out.bib')
wos_df = wos_df_filtered

Removed 0 publications from Web of Science CORE


In [28]:
# Google Scholar does not provide metadata for this
print('Removed 0 publications from Google Scholar')
export_to_csv(gs_df, './data/filter3_PublicationType/gs_filter_pubtype_in.csv')
export_to_csv(get_filtered_out_entries(gs_df, gs_df), './data/filter3_PublicationType/gs_filter_pubtype_out.csv')

Removed 0 publications from Google Scholar


In [29]:
print('-----------------------------------------------------------------------')
print(f'Removed a total of {num_before - get_num_entries()} publications that were not written fully in English')
print(f'  => {get_num_entries()} entries left in total')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Removed a total of 175 publications that were not written fully in English
  => 7038 entries left in total
-----------------------------------------------------------------------


## Filter 4: Reviews

Reviews are not publications about new technologies meaning they are not useful
for scoping reviews. Therefore, all publications that resemble any type of review
(Meta-Analysis, Systematic Review, Literature Review, ...) are filtered out.

This has to be done manually, as just looking through tags to see which article
is classified this way is insufficient. Not all articles are correctly labeled.
Additionally, looking through all titles and abstracts for the words like
"review" is prone to errors, as some publications can be part review part new
solution.

In [30]:
num_before = get_num_entries()

def filter_out_reviews(df: pd.DataFrame, title_key: str) -> pd.DataFrame:
  keywords = ['Meta-Analysis', 'Systematic Literature Review',
              'Systematic Review', 'Literature Review', 'Scoping Review',
              'Rapid Review', 'Umbrella Review', 'Narrative Review',
              'Mapping Review', 'Critical Review', 'Protocol', 'Meta-Review',
              'Analytic Review', 'Review and Analysis', 'Analysis and Review']
  pattern = '|'.join(keywords)  # regex pattern
  matches = df[title_key].str.contains(pattern, case=False, na=False)
  filtered_df = df[~matches]
  return filtered_df

In [31]:
acm_df_filtered = filter_out_reviews(acm_df, 'title')
acm_df_excluded = get_filtered_out_entries(acm_df, acm_df_filtered)
print(f'Removed {(acm_df_excluded.shape[0])} publications from ACM Digital Library')
export_to_bib(acm_df_filtered, './data/filter4_Reviews/acm_filter_rev_in.bib')
export_to_bib(acm_df_excluded, './data/filter4_Reviews/acm_filter_rev_out.bib')
acm_df = acm_df_filtered

Removed 21 publications from ACM Digital Library


In [32]:
apa_df_filtered = filter_out_reviews(apa_df, 'primary_title')
apa_df_excluded = get_filtered_out_entries(apa_df, apa_df_filtered)
print(f'Removed {(apa_df_excluded.shape[0])} publications from APA PsycInfo')
export_to_ris(apa_df_filtered, './data/filter4_Reviews/apa_filter_rev_in.ris')
export_to_ris(apa_df_excluded, './data/filter4_Reviews/apa_filter_rev_out.ris')
apa_df = apa_df_filtered

Removed 179 publications from APA PsycInfo


In [33]:
ieee_df_filtered = filter_out_reviews(ieee_df, 'title')
ieee_df_excluded = get_filtered_out_entries(ieee_df, ieee_df_filtered)
print(f'Removed {(ieee_df_excluded.shape[0])} publications from IEEE Xplore')
export_to_ris(ieee_df_filtered, './data/filter4_Reviews/ieee_filter_rev_in.ris')
export_to_ris(ieee_df_excluded, './data/filter4_Reviews/ieee_filter_rev_out.ris')
ieee_df = ieee_df_filtered

Removed 5 publications from IEEE Xplore


In [34]:
pubmed_df_filtered = filter_out_reviews(pubmed_df, 'Title')
pubmed_df_excluded = get_filtered_out_entries(pubmed_df, pubmed_df_filtered)
print(f'Removed {(pubmed_df_excluded.shape[0])} publications from PubMed & MEDLINE')
export_to_csv(pubmed_df_filtered, './data/filter4_Reviews/pubmed_filter_rev_in.csv')
export_to_csv(pubmed_df_excluded, './data/filter4_Reviews/pubmed_filter_rev_out.csv')
pubmed_df = pubmed_df_filtered

Removed 794 publications from PubMed & MEDLINE


In [35]:
wos_df_filtered = filter_out_reviews(wos_df, 'title')
wos_df_excluded = get_filtered_out_entries(wos_df, wos_df_filtered)
print(f'Removed {(wos_df_excluded.shape[0])} publications from Web of Science CORE')
export_to_bib(wos_df_filtered, './data/filter4_Reviews/wos_filter_rev_in.bib')
export_to_bib(wos_df_excluded, './data/filter4_Reviews/wos_filter_rev_out.bib')
wos_df = wos_df_filtered

Removed 135 publications from Web of Science CORE


In [36]:
gs_df_filtered = filter_out_reviews(gs_df, 'Title')
gs_df_excluded = get_filtered_out_entries(gs_df, gs_df_filtered)
print(f'Removed {(gs_df_excluded.shape[0])} publications from Google Scholar')
export_to_csv(gs_df_filtered, './data/filter4_Reviews/gs_filter_rev_in.csv')
export_to_csv(gs_df_excluded, './data/filter4_Reviews/gs_filter_rev_out.csv')
gs_df = gs_df_filtered

Removed 60 publications from Google Scholar


In [37]:
print('-----------------------------------------------------------------------')
print(f'Removed a total of {num_before - get_num_entries()} publications that had some sort of review-tag in their primary title')
print(f'  => {get_num_entries()} entries left in total')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Removed a total of 1501 publications that had some sort of review-tag in their primary title
  => 5537 entries left in total
-----------------------------------------------------------------------


## Filter 5: DOI Deduplication

The real deduplication process has to happen using some software tool with AI
or statistical help.

What can be done programatically is looking at identical DOIs. The following
code passages count how often each DOI is present across all results and remove
all entries for each DOI group across all databases until only one is left.

This naive approach introduces a subjective ordering:
  * PubMed & MEDLINE
  * ACM Digital Library
  * IEEE Xplore
  * Web of Science CORE
  * APA PsycInfo
  * Google Scholar

Entries are removed from bottom to the top, e.g. if 2 publications with the same
DOI are found, one of which being from PubMed and the other from Google Scholar,
then the entry from Google Scholar is removed. The ordering is based on how much
usable information is being provided and how easy it is to access the full-text
for each publication.

In [38]:
num_before = get_num_entries()

_doi_pattern = re.compile(r'10\.\d{4,9}/[-._;()/:A-Z0-9]+', re.IGNORECASE)
def extract_doi(doi: any) -> str:
  '''Given a DOI string or a "https://dx.doi.org/<DOI>" string this returns
     only the relevant DOI substring'''
  if pd.isna(doi) or len(str(doi)) < 5:
    return ''
  match = _doi_pattern.search(str(doi))
  return match.group(0).lower() if match else str(doi).lower()


# make all DOIs uniform and value only
acm_df['doi'] = acm_df['doi'].apply(extract_doi)
apa_df['doi'] = apa_df['doi'].apply(extract_doi)
ieee_df['doi'] = ieee_df['doi'].apply(extract_doi)
pubmed_df['DOI'] = pubmed_df['DOI'].apply(extract_doi)
wos_df['doi'] = wos_df['doi'].apply(extract_doi)
gs_df['DOI'] = gs_df['DOI'].apply(extract_doi)

In [39]:
# ordered list of dfs with the corresponding keys for the DOI columns
dfs = [(pubmed_df, 'DOI'),
       (acm_df, 'doi'),
       (ieee_df, 'doi'),
       (wos_df, 'doi'),
       (apa_df, 'doi'),
       (gs_df, 'DOI')]
# dataframe indexes have to be resetted before counting duplicates
for df, _ in dfs:
  df.reset_index(drop=True, inplace=True)
# copies must be made after index reset, otherwise comparing indices is useless
dfs_copy = [pubmed_df.copy(), acm_df.copy(), ieee_df.copy(), wos_df.copy(),
            apa_df.copy(), gs_df.copy()]

# dict to count all DOI occurrences, all DOI keys with value >1 are duplicates
doi_counts = defaultdict(int)

# finding duplicate dois across all dataframes
for i, data in enumerate(dfs):
  df, doi_key = data
  for doi in df[doi_key]:
    doi_clean = extract_doi(str(doi)).lower()
    doi_counts[doi_clean] += 1
duplicate_dois = [(doi, count) for doi, count in doi_counts.items() if pd.notna(doi) and count > 1 and len(doi) > 5]

print(f'Found {sum(count for _, count in duplicate_dois)} total duplicates')
print(f'Must keep at least {len(duplicate_dois)} of those')
print(f'=> Should remove {sum(count for _, count in duplicate_dois) - len(duplicate_dois)}')

Found 3055 total duplicates
Must keep at least 1355 of those
=> Should remove 1700


In [40]:
# list of lists, [i] = indexes in dataframes[i] to remove
indices_to_delete = {i: [] for i in range(len(dfs))}

# iterate over all duplicates
for doi, count in duplicate_dois:
  found = 0
  for df_index, (df, col) in enumerate(dfs):
    indices = df.index[df[col] == doi].tolist()
    if found == 0 and indices:
      # always keep the first found DOI row
      indices_to_delete[df_index].extend(indices[1:])
      found += len(indices)
    else:
      # if found=/=0, these new ones cannot contain the first DOI occurrance
      indices_to_delete[df_index].extend(indices)

# quick debug on what was marked for removal
n_marked_total = 0
for i, name in enumerate(['PubMed & MEDLINE', 'ACM Digital Library',
                          'IEEE Xplore', 'Web of Science CORE',
                          'APA PsycInfo', 'Google Scholar']):
  n_marked = len(indices_to_delete[list(indices_to_delete.keys())[i]])
  n_marked_total += n_marked
  print(f'Marked {n_marked} elements for removal from {name}')
print(f'=> Marked a total of {n_marked_total} elements for removal across all databases')

Marked 0 elements for removal from PubMed & MEDLINE
Marked 6 elements for removal from ACM Digital Library
Marked 6 elements for removal from IEEE Xplore
Marked 1116 elements for removal from Web of Science CORE
Marked 545 elements for removal from APA PsycInfo
Marked 27 elements for removal from Google Scholar
=> Marked a total of 1700 elements for removal across all databases


In [41]:
# dropping the marked rows
n_deleted_total = 0
for i, name in enumerate(['PubMed & MEDLINE', 'ACM Digital Library',
                          'IEEE Xplore', 'Web of Science CORE',
                          'APA PsycInfo', 'Google Scholar']):
  df = dfs[i][0]
  indices = indices_to_delete[list(indices_to_delete.keys())[i]]
  n_before = df.shape[0]
  df.drop(indices, inplace=True)
  n_after = df.shape[0]
  n_deleted = n_before - n_after
  n_deleted_total += n_deleted
  print(f'Dropped {n_deleted} elements from {name}')
print(f'=> Dropped a total of {n_deleted_total} elements for removal across all databases')

Dropped 0 elements from PubMed & MEDLINE
Dropped 6 elements from ACM Digital Library
Dropped 6 elements from IEEE Xplore
Dropped 1116 elements from Web of Science CORE
Dropped 545 elements from APA PsycInfo
Dropped 27 elements from Google Scholar
=> Dropped a total of 1700 elements for removal across all databases


In [42]:
# exporting all in and out files
acm_df_filtered = acm_df
acm_df = dfs_copy[1]
acm_df_excluded = get_filtered_out_entries(acm_df, acm_df_filtered)
print(f'Removed {(acm_df_excluded.shape[0])} publications from ACM Digital Library')
export_to_bib(acm_df_filtered, './data/filter5_DoiDeduplication/acm_filter_dedup_in.bib')
export_to_bib(acm_df_excluded, './data/filter5_DoiDeduplication/acm_filter_dedup_out.bib')
acm_df = acm_df_filtered

apa_df_filtered = apa_df
apa_df = dfs_copy[4]
apa_df_excluded = get_filtered_out_entries(apa_df, apa_df_filtered)
print(f'Removed {(apa_df_excluded.shape[0])} publications from APA PsycInfo')
export_to_ris(apa_df_filtered, './data/filter5_DoiDeduplication/apa_filter_dedup_in.ris')
export_to_ris(apa_df_excluded, './data/filter5_DoiDeduplication/apa_filter_dedup_out.ris')
apa_df = apa_df_filtered

ieee_df_filtered = ieee_df
ieee_df = dfs_copy[2]
ieee_df_excluded = get_filtered_out_entries(ieee_df, ieee_df_filtered)
print(f'Removed {(ieee_df_excluded.shape[0])} publications from IEEE Xplore')
export_to_ris(ieee_df_filtered, './data/filter5_DoiDeduplication/ieee_filter_dedup_in.ris')
export_to_ris(ieee_df_excluded, './data/filter5_DoiDeduplication/ieee_filter_dedup_out.ris')
ieee_df = ieee_df_filtered

pubmed_df_filtered = pubmed_df
pubmed_df = dfs_copy[0]
pubmed_df_excluded = get_filtered_out_entries(pubmed_df, pubmed_df_filtered)
print(f'Removed {(pubmed_df_excluded.shape[0])} publications from PubMed & MEDLINE')
export_to_csv(pubmed_df_filtered, './data/filter5_DoiDeduplication/pubmed_filter_dedup_in.csv')
export_to_csv(pubmed_df_excluded, './data/filter5_DoiDeduplication/pubmed_filter_dedup_out.csv')
pubmed_df = pubmed_df_filtered

wos_df_filtered = wos_df
wos_df = dfs_copy[3]
wos_df_excluded = get_filtered_out_entries(wos_df, wos_df_filtered)
print(f'Removed {(wos_df_excluded.shape[0])} publications from Web of Science CORE')
export_to_bib(wos_df_filtered, './data/filter5_DoiDeduplication/wos_filter_dedup_in.bib')
export_to_bib(wos_df_excluded, './data/filter5_DoiDeduplication/wos_filter_dedup_out.bib')
wos_df = wos_df_filtered

gs_df_filtered = gs_df
gs_df = dfs_copy[5]
gs_df_excluded = get_filtered_out_entries(gs_df, gs_df_filtered)
print(f'Removed {(gs_df_excluded.shape[0])} publications from Google Scholar')
export_to_csv(gs_df_filtered, './data/filter5_DoiDeduplication/gs_filter_dedup_in.csv')
export_to_csv(gs_df_excluded, './data/filter5_DoiDeduplication/gs_filter_dedup_out.csv')
gs_df = gs_df_filtered

Removed 6 publications from ACM Digital Library
Removed 545 publications from APA PsycInfo
Removed 6 publications from IEEE Xplore
Removed 0 publications from PubMed & MEDLINE
Removed 1116 publications from Web of Science CORE
Removed 27 publications from Google Scholar


In [43]:
print('-----------------------------------------------------------------------')
print(f'Removed a total of {num_before - get_num_entries()} publications had an identical DOI to at least one other publication')
print(f'  => {get_num_entries()} entries left in total')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
Removed a total of 1700 publications had an identical DOI to at least one other publication
  => 3837 entries left in total
-----------------------------------------------------------------------


## Filter 6: Manual Deduplication using Rayyan

After applying the first 5 filters, our review tool
[Rayyan](https://www.rayyan.ai/) detected additional duplicates. These are
mainly duplicates that did not get caught in the previous filter. The reasons
for that are:

* some duplicates have no DOI in their metadata
* some duplicates have different DOIs but refer to the same publication
  * e.g. 10.1007/978-3-031-21333-5 refers to the *Proceedings of the International Conference on Ubiquitous Computing & Ambient Intelligence (UCAmI 2022)*
  * e.g. 10.1007/978-3-031-21333-5_88 refers to the same publication, but specifies the correct chapter/element number
* some duplicates have different DOIs, but one is a corrigendum/correction/new version/... of the other(s)

A list of all made decisions in determining what is and is not a duplicate can
be found [here](./rayyan_deduplication.md).

The Rayyan export does not mark from which data source entries got removed. A
list of all removed entries as well as all included entries can be found in
[filter6_RayyanDeduplication](./data/filter6_RayyanDeduplication/) in form of
two CSV files.

* [articles_in](./data/filter6_RayyanDeduplication/articles_in.csv) contains all **3727** articles that are included in the abstract screening
* [articles_out](./data/filter6_RayyanDeduplication/articles_out.csv) contains all **110** articles that are excluded from the abstract screening due to being marked as duplicate of another included paper